In [7]:
%load_ext autoreload
%autoreload 2
from pathlib import Path
import neo
import neo_importers.neo_spike_importer as spike_importer
from neo_importers.neo_wrapper import MNGRecording
from features import FeatureDatabase

file_name = Path("..")/".."/"Files"/"11_10_27U2a.smr"

# maybe we could make the spike importer return a recording object immediately?
# or recording + block representation?
bl = spike_importer.import_spike_file(file_name, 
                                      stimuli_event_channels={"DigMark"},
                                      action_potential_channels={"nw-1_0#1", "nw-1_0#2", "nw-1_0#3", "nw-1_0#4", "nw-1_0#5"}
                                      )
seg: neo.core.Segment = bl.segments[0]
recording = MNGRecording(bl.segments[0])

print("There are the following channels in this recording: " + str(recording.all_channels.keys()))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Units "°C" can not be converted to a quantity. Using dimensionless instead
Units "°C" can not be converted to a quantity. Using dimensionless instead
Units "mN" can not be converted to a quantity. Using dimensionless instead
Units "mN" can not be converted to a quantity. Using dimensionless instead


There are the following channels in this recording: dict_keys(['rd.2', 'rd.1', 'rd.0', 'es.0', 'ap.0', 'ap.2', 'ap.4', 'ap.1', 'ap.3'])


In [20]:
%autoreload 2
from features import ResponseLatencyFeatureExtractor, NormalizedSignalEnergyExtractor, SpikeCountExtractor, AdaptiveSpikeCountExtractor
from quantities import second

# define feature db
db = FeatureDatabase(Path("../../Files/11_10_27U2a_Features"), recording)

try: # check if we can just load the features somehow
    db.load()
except Exception as ex: # extract the features from the data base
    ap_channels = list(recording.action_potential_channels.keys())
    db.extract_features(ap_channels, ResponseLatencyFeatureExtractor, stimulus_channel = "es.0")
    db.extract_features(ap_channels, NormalizedSignalEnergyExtractor)
    db.extract_features(ap_channels, SpikeCountExtractor, timeframe = 100 * second, num_intervals = 8)
    db.extract_features(ap_channels, AdaptiveSpikeCountExtractor, timeframe = 100 * second, num_intervals = 6)
    db.store()